In [2]:
import os
import sys
from pathlib import Path

workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("workding dir:", workding_dir)

workding dir: /Users/inflaton/code/engd/papers/chat-with-orca-2


### Translating Orca-2-13b's Results from Spanish to English

In [6]:
import pandas as pd

perf_pd1 = pd.read_excel("./results/perf_data.xlsx")
idx = (
    perf_pd1[perf_pd1["model_name"] == "orca-2-13b"]
    .groupby("model_name")["overall_score"]
    .idxmax()
)
df = perf_pd1.loc[idx].sort_index(ascending=False)
df

,model_name,repetition_penalty,faithfulness,answer_relevancy,overall_score,total_time_used,num_tokens_generated,token_per_second
6,orca-2-13b,1.05,1.0,0.987592,0.993757,397.548,641,1.612


In [7]:
selected = []
NUM_QUESTIONS =4
for j in range(NUM_QUESTIONS):
    for i in idx.values:
        selected.append(i * NUM_QUESTIONS + j)

In [14]:
raw_data = pd.read_excel("./results/raw_data.xlsx")
df = raw_data.loc[selected]
df = df.drop(["repetition_penalty", "contexts"], axis=1)
df = df.fillna("")
df[df.columns] = df.apply(lambda x: x.str.strip())
df["standalone_question"] = df["standalone_question"].str.replace("\n", "**\n**")
df["standalone_question"] = df["standalone_question"].apply(
    lambda x: "{}{}{}".format("**", x, "**") if len(x) > 0 else x
)
df["standalone_question"] = df["standalone_question"].str.replace("****", "")
df["LLM Generated Content"] = (
    df["standalone_question"].str.cat(df["answer"], sep="\n").str.strip()
)
df = df.rename(columns={"model_name": "Model Name"})
df = df.drop(columns=["answer", "standalone_question", "user_question"])
df[df.columns] = df.apply(lambda x: x.str.strip())
df.reset_index(drop=True, inplace=True)
df

,Model Name,LLM Generated Content
0,orca-2-13b,PCI DSS is a global standard that provides a b...
1,orca-2-13b,**¿Puedes resumir los cambios realizados desde...
2,orca-2-13b,**¿Cuáles son las nuevas requisitos para las e...
3,orca-2-13b,**¿Puedes dar más detalles sobre las prácticas...


In [15]:
df.drop(index=[0], inplace=True)
orca_2_13b_df = df
orca_2_13b_df["LLM Generated Content"] = orca_2_13b_df["LLM Generated Content"].apply(
    lambda x: x.replace("**", "")
)
orca_2_13b_df

,Model Name,LLM Generated Content
1,orca-2-13b,¿Puedes resumir los cambios realizados desde l...
2,orca-2-13b,¿Cuáles son las nuevas requisitos para las eva...
3,orca-2-13b,¿Puedes dar más detalles sobre las prácticas d...


In [44]:
import os
from openai import OpenAI
from app_modules.init import *


# Function to translate text using OpenAI
def translate(text):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Translate the following text to English: {text}",
            }
        ],
        model="gpt-4-turbo-preview",
    )
    print(chat_completion)
    return chat_completion.choices[0].message.content

In [58]:
orca_2_13b_df["Translation by GPT-4-Turbo"] = orca_2_13b_df[
    "LLM Generated Content"
].apply(translate)
orca_2_13b_df

ChatCompletion(id='chatcmpl-8mdHUCObrRgeCDBweZXGDc0FTVrh8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Can you summarize the changes made from version 3.2.1 to version 4.0 of PCI DSS?\nYes, I can summarize the changes made from version 3.2.1 to version 4.0 of PCI DSS. The changes include adjustments to align with the modifications from PCI DSS 3.2 to PCI DSS 3.2.1, as well as minor corrections and edits to clarify and improve the format. Then, in version 4.0, updates were made to align with the modifications from PCI DSS 3.2.1 to PCI DSS 4.0, and there were also minor corrections and edits made to clarify and improve the format. In December 2022, there was an additional revision that included minor adjustments, correction of typographical errors, and the removal of "In Place with Remediation" as a reporting option.', role='assistant', function_call=None, tool_calls=None))], created=1706601432, model='gpt-4-0125-preview', object=

,Model Name,LLM Generated Content,Translation by GPT-4-Turbo
1,orca-2-13b,¿Puedes resumir los cambios realizados desde l...,Can you summarize the changes made from versio...
2,orca-2-13b,¿Cuáles son las nuevas requisitos para las eva...,What are the new requirements for vulnerabilit...
3,orca-2-13b,¿Puedes dar más detalles sobre las prácticas d...,Can you provide more details about penetration...


In [59]:
print(orca_2_13b_df.to_markdown())

|    | Model Name   | LLM Generated Content                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Translation by GPT-4-Turbo                                                                                                                                                                                                                                                                                        

### Translation of Contents Generated by Orca-2-13b

|    | Model Name   | LLM Generated Content                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Translation by GPT-4-Turbo                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|---:|:-------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  1 | orca-2-13b   | ¿Puedes resumir los cambios realizados desde la versión 3.2.1 hasta la versión 4.0 de PCI DSS?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Can you summarize the changes made from version 3.2.1 to version 4.0 of PCI DSS?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |              | Sí, puedo resumir los cambios realizados desde la versión 3.2.1 hasta la versión 4.0 de PCI DSS. Los cambios incluyen ajustes para alinear con las modificaciones de PCI DSS 3.2 a PCI DSS 3.2.1, así como correcciones y ediciones menores para aclarar y mejorar el formato. Luego, en la versión 4.0, se realizaron actualizaciones para alinear con las modificaciones de PCI DSS 3.2.1 a PCI DSS 4.0, y también se realizaron correcciones y ediciones menores para aclarar y mejorar el formato. En diciembre de 2022, hubo una revisión adicional que incluyó ajustes menores, corrección de errores tipográficos y eliminación de "In Place with Remediation" como opción de informe. | Yes, I can summarize the changes made from version 3.2.1 to version 4.0 of PCI DSS. The changes include adjustments to align with the modifications from PCI DSS 3.2 to PCI DSS 3.2.1, as well as minor corrections and edits to clarify and improve the format. Then, in version 4.0, updates were made to align with the modifications from PCI DSS 3.2.1 to PCI DSS 4.0, and there were also minor corrections and edits made to clarify and improve the format. In December 2022, there was an additional revision that included minor adjustments, correction of typographical errors, and the removal of "In Place with Remediation" as a reporting option. |
|  2 | orca-2-13b   | ¿Cuáles son las nuevas requisitos para las evaluaciones de vulnerabilidades en la versión 4.0 de PCI DSS?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | What are the new requirements for vulnerability assessments in PCI DSS version 4.0?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|    |              | Las nuevas requisitos para las evaluaciones de vulnerabilidades en la versión 4.0 de PCI DSS incluyen resolver vulnerabilidades con una puntuación de 4.0 o más en el CVSS, realizar rescans cuando sea necesario, y tener un método de penetration testing definido, documentado e implementado que incluye enfoques aceptados por la industria y cobertura para el perímetro del SDE y los sistemas críticos.                                                                                                                                                                                                                                                                               | The new requirements for vulnerability assessments in PCI DSS version 4.0 include resolving vulnerabilities with a CVSS score of 4.0 or higher, performing rescans when necessary, and having a defined, documented, and implemented penetration testing method that includes industry-accepted approaches and coverage for the CDE perimeter and critical systems.                                                                                                                                                                                                                                                                                               |
|  3 | orca-2-13b   | ¿Puedes dar más detalles sobre las prácticas de prueba de penetración en PCI DSS?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Can you provide more details about penetration testing practices in PCI DSS?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |              | Sí, puedo dar más detalles sobre las prácticas de prueba de penetración en PCI DSS. La PCI DSS requiere que se realice un penetration testing externo al menos una vez cada 12 meses y después de cualquier modificación o cambio significativo en la infraestructura o el software. El penetration testing debe ser realizado por un recurso interno calificado o un tercero de fuera de la organización calificado y debe cumplir con la metodología definida por la entidad. Además, las vulnerabilidades y debilidades de seguridad que se encuentren durante el penetration testing deben ser corregidas y verificadas mediante la repetición del testing.                               | Yes, I can provide more details about penetration testing practices in PCI DSS. The PCI DSS requires that external penetration testing be conducted at least once every 12 months and after any significant modification or change in the infrastructure or software. The penetration testing must be conducted by a qualified internal resource or a qualified third party from outside the organization and must comply with the methodology defined by the entity. Additionally, the security vulnerabilities and weaknesses that are found during the penetration testing must be corrected and verified by repeating the testing.   